In [1]:
from ViT_Model import create_model

In [3]:
import torch
import numpy as np
from torchvision import transforms

device = torch.device("cuda:0")

In [4]:
image_size = 336
resize_to = int(image_size * 1.125)
ppc_fn = transforms.Compose([
            transforms.Resize((resize_to, resize_to),  interpolation=3),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.4850, 0.4560, 0.4060), (0.2290, 0.2240, 0.2250)),
            transforms.ConvertImageDtype(torch.float16)
        ])

In [5]:
model = create_model(check_pth="/media/Data/networks/Pytorch/CLIP-FT/GPRet_ViTL.pth")
model.to(device)
model.eval();

In [15]:
example_input = torch.rand(1, 3, 336, 336).half().to(device)
out = model(example_input)

In [16]:
out.shape

torch.Size([1, 1024])

In [19]:
# binary vectors
bv = (out < 0).int()

In [20]:
bv.shape

tensor([[0, 0, 0,  ..., 1, 0, 0]], device='cuda:0', dtype=torch.int32)

In [21]:
bv.shape

torch.Size([1, 1024])

In [22]:
# JIT Version erstellen
gpr_vit_jit = torch.jit.trace(model, example_input).to(torch.float16)

/home/administrator/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:4135: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert embed_dim == embed_dim_to_check
/home/administrator/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:4137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert key.size(0) == value.size(0) and key.size(1) == value.size(1)
/home/administrator/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:4140: TracerWarning: Converting a tensor to a Python boolean mig